# 아웃바운드 인증

아웃바운드 인증을 통해 에이전트와 AgentCore Gateway는 인바운드 인증 과정에서 인증 및 권한이 부여된 사용자를 대신하여 AWS 리소스 및 타사 서비스에 안전하게 액세스할 수 있습니다. AWS 리소스 또는 타사 서비스와 권한 부여를 통합하려면 인바운드 인증과 아웃바운드 인증을 모두 구성해야 합니다.

AgentCore Identity가 지원하는 적정 수준의 액세스 및 안전한 권한 위임을 통해 에이전트는 AWS 리소스 및 GitHub, Google, Salesforce, Slack과 같은 타사 도구에 원활하고 안전하게 액세스할 수 있습니다. 에이전트는 사용자 대신 또는 사전 승인된 사용자 동의가 있는 경우 독립적으로 이러한 서비스에서 작업을 수행할 수 있습니다. 또한, 보안 토큰 저장소를 사용하여 동의 피로도를 줄이고 간소화된 AI 에이전트 환경을 구축할 수 있습니다.

## 아웃바운드 인증 구성

먼저 클라이언트 애플리케이션을 타사 공급업체에 등록한 다음 아웃바운드 인증을 생성합니다. AWS 리소스, 타사 서비스 또는 AgentCore Gateway 대상에 대한 액세스를 검증하는 방법을 지정합니다. OAuth 2LO/3LO 또는 API 키를 사용할 수 있습니다. OAuth를 사용하면 AgentCore Identity에서 제공하는 공급자 중에서 선택할 수 있습니다. 이 경우 AgentCore Identity에서 공급자의 구성 세부 정보를 입력합니다. 또는 사용자 지정 공급자의 세부 정보를 제공할 수도 있습니다.

사용자가 AWS 리소스, 타사 서비스 또는 AgentCore Gateway 대상에 액세스하려는 경우, 아웃바운드 인증은 수신 인증에서 제공한 액세스 토큰이 유효한지 확인하고, 유효한 경우 해당 리소스에 대한 액세스를 허용합니다.

<div style="text-align:center">
<img src="images/outbound_auth.png" width="90%"/>
</div>

## 리소스 자격 증명 공급자

이 구성 요소는 에이전트 코드가 다운스트림 리소스 서버(예: Google, GitHub)의 자격 증명을 검색하여 액세스하는 데 사용하는 구성 요소입니다. 예를 들어 Gmail에서 이메일을 가져오거나 Google 캘린더에 회의를 추가하는 등의 작업을 수행할 수 있습니다. AgentCore는 최종 사용자, 에이전트 코드 및 외부 권한 부여 서버 전반에 걸쳐 2LO 및 3LO OAuth2 오케스트레이션 흐름을 구현하는 에이전트 개발자의 번거로운 작업을 제거합니다. AgentCore는 사용자 지정 OAuth2 자격 증명 공급자와 Google, GitHub, Slack, Salesforce와 같은 기본 제공 공급자 목록을 제공하며, 권한 부여 서버 엔드포인트와 공급자별 매개변수가 미리 입력되어 있습니다.

Bedrock AgentCore Identity는 에이전트 개발자가 OAuth2 또는 API 키를 지원하는 외부 리소스에 인증할 수 있도록 OAuth2 및 API 키 자격 증명 공급자를 제공합니다. 다음 예시에서는 API 키 자격 증명 공급자를 구성하는 방법을 안내합니다. 에이전트는 API 키 자격 증명 공급자를 사용하여 모든 에이전트 작업에 필요한 API 키를 검색할 수 있습니다. 다른 자격 증명 공급자에 대한 설명서는 해당 문서를 참조하십시오.

### 리소스 자격 증명 공급자 생성.

다음은 API 키 리소스 자격 증명 공급자를 생성하는 예시입니다.

```
bedrock_agentcore.services.identity에서 IdentityClient 가져오기
identity_client = IdentityClient(region="us-west-2")

api_key_provider = identity_client.create_api_key_credential_provider({
"name": "APIKey-provider-name",
"apiKey": "<my-api-key>" # 외부 애플리케이션 공급업체(예: OpenAI)에서 얻은 API 키로 대체
})
print(api_key_provider)
```

### 리소스 자격 증명 공급자에서 액세스 토큰 또는 API 키를 가져옵니다.

다음은 API 키 자격 증명 공급자에서 API 키를 가져오는 예입니다. 에이전트는 API 키를 사용하여 LLM 또는 API 키 구성을 사용하는 다른 서비스와 상호 작용할 수 있습니다. 자격 증명 공급자에서 액세스 토큰 또는 API 키와 같은 자격 증명을 가져오려면 아래와 같이 함수를 데코레이팅할 수 있습니다.

```
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key

@requires_api_key(
provider_name="APIKey-provider" # 사용자 인증 정보 공급자 이름으로 변경
)
async def need_api_key(*, api_key: str):
print(f'received api key for async func: {api_key}')

await need_api_key(api_key="")
```

@require_access_token 데코레이터와 함께 사용할 수 있는 다양한 매개변수는 다음과 같습니다.| 매개변수 이름 | 설명 |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name | 자격 증명 공급자 이름 |
| into | 토큰을 주입할 매개변수 이름 |
| scopes | 요청할 OAuth2 범위 |
| on_auth_url | 권한 부여 URL 처리를 위한 콜백 |
| auth_flow | 인증 흐름 유형("M2M" 또는 "USER_FEDERATION") |
| callback_url | OAuth2 콜백 URL |
| force_authentication| 강제 재인증 |
| token_poller | 사용자 지정 토큰 폴러 구현 |

# Amazon Bedrock AgentCore 런타임에서 OpenAI 모델을 사용하는 스트랜드 에이전트 호스팅

## 개요

이 튜토리얼에서는 01-AgentCore-runtime에서 배포한 에이전트를 수정하여 openai-model을 사용하고 API 키 자격 증명 공급자를 사용하여 아웃바운드 인증을 구성합니다. API 키 자격 증명 공급자를 설정하여 OpenAI 키를 저장하고, 이 키를 사용하도록 에이전트 코드를 수정합니다.

### 튜토리얼 아키텍처

<div style="text-align:center">
<img src="images/outbound_auth_api.png" width="90%"/>
</div>

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:-------------------------------------------------------------------------|
| 튜토리얼 유형 | 대화형 |
| 에이전트 유형 | 단일 |
| 에이전트 프레임워크 | 스트랜드 에이전트 |
| LLM 모델 | GPT 4.1 미니 |
| 튜토리얼 구성 요소 | AgentCore 런타임에서 에이전트 호스팅. 스트랜드 에이전트 및 OpenAI 모델 사용 |
| 튜토리얼 수직 | 교차 수직 |
| 예시 복잡성 | 쉬움 |
| 사용 SDK | Amazon BedrockAgentCore Python SDK 및 boto3 |
| 자격 증명 공급자 | 유형: API 키 |

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore 런타임에서 에이전트 호스팅
* OpenAI 모델 사용
* Strands 에이전트 사용
* API 키 자격 증명 공급자를 통한 AgentCore 송신 인증 사용.

## 필수 조건

이 튜토리얼을 실행하려면 다음이 필요합니다.
* Python 3.10 이상
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands 에이전트
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## 에이전트 생성 및 로컬 실험

AgentCore Runtime에 에이전트를 배포하기 전에, 실험을 위해 로컬에서 개발하고 실행해 보겠습니다.

운영 에이전트 애플리케이션의 경우, 에이전트 생성 프로세스와 에이전트 호출 프로세스를 분리해야 합니다. AgentCore Runtime에서는 에이전트의 호출 부분을 `@app.entrypoint` 데코레이터로 데코레이팅하고 이를 런타임의 진입점으로 설정합니다. 먼저 실험 단계에서 각 에이전트가 어떻게 개발되는지 살펴보겠습니다.

아키텍처는 다음과 같습니다.

<div style="text-align:left">
<img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os

##Update the below configuration with your Azure API Key details.
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

#### Invoking local agent

In [ ]:
!python strands_agents_openai.py '{"prompt": "What is the weather now?"}'

## Create a Resource Credential Provider

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

from boto3.session import Session
import boto3
boto_session = Session()
region = boto_session.region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "openai-apikey-provider",
    "apiKey": "<YOUR_API_KEY>" # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
})
print(api_key_provider)


## AgentCore 런타임에 에이전트 배포 준비 및 리소스 자격 증명 공급자 사용

이제 AgentCore 런타임에 에이전트를 배포해 보겠습니다. 이를 위해 다음 작업을 수행해야 합니다.
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` 명령어를 사용하여 런타임 앱을 가져옵니다.
* `app = BedrockAgentCoreApp()` 명령어를 사용하여 코드에서 앱을 초기화합니다.
* `@app.entrypoint` 데코레이터를 사용하여 호출 함수를 데코레이팅합니다.
* `app.run()` 명령어를 사용하여 AgentCoreRuntime이 에이전트 실행을 제어하도록 합니다.
* 이전 단계에서 생성한 리소스 자격 증명 공급자에서 OpenAI 키를 가져옵니다.

### OpenAI 모델을 사용한 Strands 에이전트
GPT 4.1 미니 모델을 사용하여 Strands 에이전트부터 시작해 보겠습니다. 다른 모든 모델은 동일하게 작동합니다.

In [ ]:
%%writefile strands_agents_openai.py
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key
from strands import Agent, tool
from strands_tools import calculator 
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

AZURE_API_KEY_FROM_CREDS_PROVIDER = ""


@requires_api_key(
    provider_name="openai-apikey-provider" # replace with your own credential provider name
)
async def need_api_key(*, api_key: str):
    global AZURE_API_KEY_FROM_CREDS_PROVIDER
    print(f'received api key for async func: {api_key}')
    AZURE_API_KEY_FROM_CREDS_PROVIDER = api_key

# Don't print empty value at module level - will print in entrypoint function

app = BedrockAgentCoreApp()

# API key will be set dynamically in the entrypoint function
#Update the below configuration with your Azure API Key details.
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

# Global agent variable
agent = None

@app.entrypoint
async def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    global AZURE_API_KEY_FROM_CREDS_PROVIDER, agent
    
    print(f"Entrypoint called with AZURE_API_KEY_FROM_CREDS_PROVIDER: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
    
    # Get API key if not already retrieved
    if not AZURE_API_KEY_FROM_CREDS_PROVIDER:
        print("Attempting to retrieve API key...")
        try:
            await need_api_key(api_key="")
            print(f"API key retrieved: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
            os.environ["AZURE_API_KEY"] = AZURE_API_KEY_FROM_CREDS_PROVIDER
            print("Environment variable AZURE_API_KEY set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("API key already available")
    
    # Initialize agent after API key is set
    if agent is None:
        print("Initializing agent with API key...")
        model = "azure/gpt-4.1-mini"
        litellm_model = LiteLLMModel(
            model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
        )
        
        agent = Agent(
            model=litellm_model,
            tools=[calculator, weather],
            system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
        )
        print("Agent initialized successfully")
    
    user_input = payload.get("prompt")
    print(f"User input: {user_input}")
    
    try:
        response = agent(user_input)
        print(f"Agent response: {response}")
        return response.message['content'][0]['text']
    except Exception as e:
        print(f"Error in agent processing: {e}")
        raise

if __name__ == "__main__":
    app.run()


## 내부적으로는 무슨 일이 일어날까요?

`BedrockAgentCoreApp`을 사용하면 자동으로 다음 작업이 수행됩니다.

* 8080 포트에서 수신 대기하는 HTTP 서버를 생성합니다.
* 에이전트의 요구 사항을 처리하는 데 필요한 `/invocations` 엔드포인트를 구현합니다.
* 상태 확인을 위한 `/ping` 엔드포인트를 구현합니다(비동기 에이전트에 매우 중요).
* 적절한 콘텐츠 유형 및 응답 형식을 처리합니다.
* AWS 표준에 따라 오류 처리를 관리합니다.

## AgentCore 런타임에 에이전트 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 에이전트와 통신하는 방식을 제어할 수 있습니다.

**참고:** 작업 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다.

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore 런타임에 배포할 수 있습니다.

### AgentCore 런타임 배포 구성

다음으로, 시작 툴킷을 사용하여 진입점, 방금 생성한 실행 역할, 그리고 요구 사항 파일을 사용하여 AgentCore 런타임 배포를 구성합니다. 또한 시작 키트가 실행 시 Amazon ECR 저장소를 자동으로 생성하도록 구성합니다.

구성 단계에서는 애플리케이션 코드를 기반으로 Docker 파일이 생성됩니다.

<div style="text-align:left">
<img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name="strands_agents_openai"

response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    region=region
)
response

### AgentCore 런타임에 에이전트 실행

이제 Docker 파일이 생성되었으니 AgentCore 런타임에 에이전트를 실행해 보겠습니다. 이렇게 하면 Amazon ECR 저장소와 AgentCore 런타임이 생성됩니다.

<div style="text-align:left">
<img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

#### 자동 생성된 역할에 필요한 추가 정책 추가

에이전트에 아웃바운드 ID를 추가하기 때문에 자동 생성된 역할에서 사용할 수 없는 API 키와 비밀번호를 가져와야 합니다. 이를 위해 자동 생성된 IAM 역할에 추가 권한을 추가해야 합니다. 먼저 이 역할을 가져온 다음 해당 권한을 추가해 보겠습니다.

In [ ]:
import json
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore 런타임 상태 확인
이제 AgentCore 런타임을 배포했으니 배포 상태를 확인해 보겠습니다.

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore 런타임 호출

마지막으로, 페이로드를 사용하여 AgentCore 런타임을 호출할 수 있습니다.

<div style="text-align:left">
<img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hello"}, user_id="userid_1234567890")
invoke_response

### 호출 결과 처리

이제 호출 결과를 처리하여 애플리케이션에 포함할 수 있습니다.

In [ ]:
from IPython.display import Markdown, display
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### boto3를 사용하여 AgentCore 런타임 호출하기

이제 AgentCore 런타임이 생성되었으므로 모든 AWS SDK를 사용하여 호출할 수 있습니다. 예를 들어 boto3의 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeUserId="userid_1234567890",
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=runtime_role.split("/")[1],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=runtime_role.split("/")[1],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=runtime_role.split("/")[1]
)

# Congratulations!